<a href="https://colab.research.google.com/github/wisenutgolead/wisenut_NER/blob/master/make_input_data_using_word_pos_entity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

json 데이터 가져오기


In [0]:
import json
import re

# json 파일 열기
# json 파일 경로는 알아서 수정
with open('/content/drive/My Drive/와이즈넛/RESULT/annotations/chosj/2013_reviewed_deleted_1.txt.adm.json', 'r', encoding='utf-8') as f:
  json_data = json.load(f)

datas = json_data['data'] # 문장들만 가져오기
attributes = json_data['attributes']['entities']['items'] # 단어의 정보를 담고 있는 딕셔너리들을 가져온다

# attributes 순서를 전체 문장에서 나오는 순서로 바꾼다(start index 기준으로 오름차순 정렬)
attributes = sorted(attributes, key=lambda x : x['mentions'][0]['startOffset'])


# 함수
def get_start_index(i):
  return attributes[i]['mentions'][0]['startOffset']

def get_end_index(i):
  return attributes[i]['mentions'][0]['endOffset']

def get_entity(i):
  return attributes[i]['type']


entities = [get_entity(i) for i in range(len(attributes))] # 문서에 있는 모든 entity(문서에 나오는 순서로 저장되어 있다)
entity_words = [datas[get_start_index(i):get_end_index(i)].strip().replace(' ', '') for i in range(len(attributes))] # entity인 단어

sentences = re.split('\r\n|\u3000', datas) # 정규표현식을 이용해 문서를 문장으로 나눈다
sentences = [x for x in sentences if len(x) != 0] # 길이가 0인 문장 제거

mecab 설치</br></br>
코랩(리눅스)에서만 설치 가능. 윈도우 환경에서 설치하려면 [여기](https://cleancode-ws.tistory.com/97) 참고

In [0]:
!pip install konlpy

In [0]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

mecab 으로 문장을 형태소로 분해

In [0]:
from konlpy.tag import Mecab
mecab = Mecab()

mecab_sentences = [] # mecab으로 형태소 분석 된 문장을 저장

def get_pos(sentences): 
  for sentence in sentences:
    mecab_sentences.append(mecab.pos(sentence))

get_pos(sentences)

형태소로 분해된 단어를 기반으로 entity 태그 붙혀서 input data 만들기

In [0]:
word_pos_entity_all = []

idx = 0 # entity로 취급되는 단어 리스트에서 단어를 가져오기 위해 사용하는 인덱스

for mecab_sentence in mecab_sentences: # 전체 문장
  
  word_pos_entity = []
  word_len = 0 # 형태소 분해된 단어의 길이
  entity_len = 0 # entity 단어의 길이

  for word, pos in mecab_sentence: # 문장
  
    if word in entity_words[idx]: # 형태소가 entity 단어에 포함되면
      # 형태소와 entity 단어의 길이를 잰다
      word_len += len(word)
      entity_len = len(entity_words[idx])
      word_pos_entity.append([word, pos, 'I-'+entities[idx]])

      # 형태소의 길이와 entity 단어의 길이가 같아지면 인덱스가 다음 entity 단어를 가리키게 +1을 해준다
      if word_len == entity_len: 
        word_len = 0
        entity_len = 0
        idx += 1
    # 형태소가 entity가 아닌 경우
    else:
      word_pos_entity.append([word, pos, 'O']) 
  
  word_pos_entity_all.append(word_pos_entity)

만든 데이터 확인

In [0]:
word_pos_entity_all[0]